In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
import os
import pickle
import imageio

In [ ]:
# beta controls the sharpness of change.
# for sudden change, set beta to a large value
def sigmoid(t, t0, beta):
    kernel = -beta*(t-t0)
    kernel = np.clip(kernel, -600, 600)
    return 1./(1. + np.exp(kernel))
def d_sigmoid(t, t0, beta):
    tmp = sigmoid(t, t0, beta)
    return tmp*(1-tmp)*beta

In [ ]:
def gen_circle(nx, ny, xo, yo, r, cf, cb):
    ## nx, ny : size of figure
    ## xo, yo : pos of center
    ## r : radius
    ## cf, cb: color, grey from and back
    a = cb*np.ones((ny, nx))
    for i in range(ny):
        for j in range(nx):
            xi = (j+0.5)/nx
            yi = (ny-i-0.5)/ny
            #print(i, j, xi, yi)
            d2 = (xi-xo)**2 + (yi-yo)**2
            if(d2<r*r):
                a[i,j] = cf
    return a

def gen_rec(nx, ny, xo, yo, ax, ay, cf, cb):
    ## nx, ny : size of figure
    ## xo, yo : pos of center
    ## ax, ay : size x, size y
    ## c: color, grey
    a = cb*np.ones((ny, nx))
    for i in range(ny):
        for j in range(nx):
            xi = (j+0.5)/nx
            yi = (ny-i-0.5)/ny
            dis_x = np.abs(xi-xo)
            dis_y = np.abs(yi-yo)
            if(dis_x<ax and dis_y<ay):
                a[i,j] = cf
    return a

def gen_holo(nx, ny, xo, yo, r1, r2, cf, cb):
    ## nx, ny : size of figure
    ## xo, yo : pos of center
    ## r1, r2 : inner, outer size
    ## c: color, grey
    a = cb*np.ones((ny, nx))
    for i in range(ny):
        for j in range(nx):
            xi = (j+0.5)/nx
            yi = (ny-i-0.5)/ny
            d2 = (xi-xo)**2 + (yi-yo)**2
            if(d2<r2*r2 and d2>=r1*r1):
                a[i,j] = cf
    return a

def noize_mixer(a, level, dist):
    noize = None
    if(not dist in ["uniform", "gaussian"]):
        print("Error: Unknown noize type!")
        return None
    if(dist == "uniform"):
        noize = level*np.random.uniform(-1.0, 1.0, a.shape)
    if(dist == "gaussian"):
        noize = level*np.random.normal(0.0, 1.0, a.shape)
    
    a1 = a+noize
    a1 = np.maximum(a1, 0.0)
    a1 = np.minimum(a1, 1.0)
    
    return a1

def trans_int(a):
    return (255.0*a).astype(np.uint8)

In [ ]:
def gen_picture_dataset(dimx, dimy, n, alpha, beta, t0, noize_level, folder_name, rand_pos=True):
    ## define some const
    
    #dimx = 64
    #dimy = 64
    r0 = 0.3
    r11 = 0.15
    r12 = 0.30
    uax = 0.15
    uay = 0.20
    cf0 = 0.8; cf1 = 0.8; cfu = 0.8; # face color
    cb0 = 0.2; cb1 = 0.2; cbu = 0.2; # background color
    
    meta_data = {}
    
    n_01 = int(n*alpha)
    n_u = n - n_01
    
    count = 0
    pic_type_count = {
        (False, "u"):0,
        (True, "u"):0,
        (False, "0"):0,
        (True, "0"):0,
        (False, "1"):0,
        (True, "1"):0,
    }
    
    for i in range(n_u):
        ## random generate t
        t = np.random.uniform(0.0, 1.0)
        pic_type_count[(t>t0,"u")]+=1
        
        ## random generate picture
        uax_i = uax; uay_i = uay;
        rg_u_x = [uax_i, 1.0-uax_i]
        rg_u_y = [uay_i, 1.0-uay_i]
        
        if(rand_pos):
            x0_i = np.random.uniform(rg_u_x[0], rg_u_x[1])
            y0_i = np.random.uniform(rg_u_y[0], rg_u_y[1])
        else:
            x0_i = 0.5
            y0_i = 0.5
        
        pic = gen_rec(dimx,dimy,x0_i,y0_i,uax_i,uay_i,cfu,cbu)
        pic = noize_mixer(pic, noize_level, "gaussian")
        pic = trans_int(pic)
        
        picid = "%010d"%(count)
        entry = {"t":t, "cf":cfu, "cb":cbu, "dim1":uax_i, "dim2":uay_i}
        meta_data[picid] = entry
        fname = folder_name+"/"+picid+".png"
        imageio.imwrite(fname, pic)
        count+=1;
        if(count%1000==0): print("Generated %10d/%10d"%(count, n))
        
        #entry = {"pic":pic, "t":t, "cf":cfu, "cb":cbu, "dim1":uax_i, "dim2":uay_i}
        #entry_meta = np.array([t, cfu, cbu, uax_i, uay_i])
        #entry_pic = pic.flatten()
        #entry = np.append(entry_meta, entry_pic)
        #data_block.append(entry)
        
    for i in range(n_01):
        ## random generate t
        t = np.random.uniform(0.0, 1.0)
        p1 = sigmoid(t, t0, beta)
        decide = np.random.uniform(0,1)
        if(decide <= p1):
            pic_type_count[(t>t0,"1")]+=1
            
            r11_i = r11; r12_i = r12;
            rg_1_x = [r12_i, 1.0-r12_i]
            rg_1_y = [r12_i, 1.0-r12_i]
            
            if(rand_pos):
                x0_i = np.random.uniform(rg_1_x[0], rg_1_x[1])
                y0_i = np.random.uniform(rg_1_y[0], rg_1_y[1])
            else:
                x0_i = 0.5
                y0_i = 0.5
            
            pic = gen_holo(dimx,dimy,x0_i,y0_i,r11_i,r12_i,cf1,cb1)
            pic = noize_mixer(pic, noize_level, "gaussian")
            pic = trans_int(pic)
            
            picid = "%010d"%(count)
            entry = {"t":t, "cf":cf1, "cb":cb1, "dim1":r11_i, "dim2":r12_i}
            meta_data[picid] = entry       
            fname = folder_name+"/"+picid+".png"
            imageio.imwrite(fname, pic)
            count+=1;
            if(count%1000==0): print("Generated %10d/%10d"%(count, n))
            
            #entry_meta = np.array([t, cf1, cb1, r11_i, r12_i])
            #entry_pic = pic.flatten()
            #entry = np.append(entry_meta, entry_pic)
            #data_block.append(entry)
        else:
            pic_type_count[(t>t0,"0")]+=1
            
            r0_i = r0
            rg_0_x = [r0_i, 1.0-r0_i]
            rg_0_y = [r0_i, 1.0-r0_i]
            
            if(rand_pos):
                x0_i = np.random.uniform(rg_0_x[0], rg_0_x[1])
                y0_i = np.random.uniform(rg_0_y[0], rg_0_y[1])
            else:
                x0_i = 0.5
                y0_i = 0.5
            
            pic = gen_circle(dimx, dimy, x0_i, y0_i, r0_i, cf0, cb0)
            pic = noize_mixer(pic, noize_level, "gaussian")
            pic = trans_int(pic)
            
            picid = "%010d"%(count)
            entry = {"t":t, "cf":cf0, "cb":cb0, "dim1":r0_i, "dim2":r0_i}
            meta_data[picid] = entry
            fname = folder_name+"/"+picid+".png"
            imageio.imwrite(fname, pic)
            count+=1;
            if(count%1000==0): print("Generated %10d/%10d"%(count, n))            
            #entry_meta = np.array([t, cf1, cb1, r0_i, r0_i])
            #entry_pic = pic.flatten()
            #entry = np.append(entry_meta, entry_pic)
            #data_block.append(entry)
    
    print("Summary:")
    print("%8s %8s %8s %8s"%(" ", "u", "0", 1))
    print("%8s %8d %8d %8d"%("t<t0",\
        pic_type_count[(False,"u")],\
        pic_type_count[(False,"0")],\
        pic_type_count[(False,"1")],\
        ))
    print("%8s %8d %8d %8d"%("t>t0",\
        pic_type_count[(True,"u")],\
        pic_type_count[(True,"0")],\
        pic_type_count[(True,"1")],\
        ))
    
    #data_block = np.array(data_block)
    #pickle.dump(data_block, open(fname, 'wb'))
    fname = folder_name+"/"+"meta_data.bin"
    pickle.dump(meta_data, open(fname, 'wb'))
    print("Metadata saved.")
    print("Done.")

In [ ]:
def create_demostration_nosquare(n_seg, beta, t0, noize_level, file_name, rand_pos=True, show_t=True):
    ## define some const
    
    dimx = 64
    dimy = 64
    r0 = 0.3
    r11 = 0.15
    r12 = 0.30
    uax = 0.15
    uay = 0.20
    cf0 = 0.8; cf1 = 0.8; cfu = 0.8; # face color
    cb0 = 0.2; cb1 = 0.2; cbu = 0.2; # background color
    
    pic_01 = []
    pic_u = []

    for i in range(n_seg//2):
        r0_i = r0
        rg_0_x = [r0_i, 1.0-r0_i]
        rg_0_y = [r0_i, 1.0-r0_i]
        
        x0_i = np.random.uniform(rg_0_x[0], rg_0_x[1])
        y0_i = np.random.uniform(rg_0_y[0], rg_0_y[1])
        pic = gen_circle(dimx, dimy, x0_i, y0_i, r0_i, cf0, cb0)
        pic = noize_mixer(pic, noize_level, "gaussian")
        pic = trans_int(pic)
        pic_01.append(pic)
        
    for i in range(n_seg//2):
        r11_i = r11; r12_i = r12;
        rg_1_x = [r12_i, 1.0-r12_i]
        rg_1_y = [r12_i, 1.0-r12_i]
        
        x0_i = np.random.uniform(rg_1_x[0], rg_1_x[1])
        y0_i = np.random.uniform(rg_1_y[0], rg_1_y[1])
        pic = gen_holo(dimx,dimy,x0_i,y0_i,r11_i,r12_i,cf1,cb1)
        pic = noize_mixer(pic, noize_level, "gaussian")
        pic = trans_int(pic)
        pic = pic_01.append(pic)
    
    for i in range(n_seg):
        uax_i = uax; uay_i = uay;
        rg_u_x = [uax_i, 1.0-uax_i]
        rg_u_y = [uay_i, 1.0-uay_i]
        
        x0_i = np.random.uniform(rg_u_x[0], rg_u_x[1])
        y0_i = np.random.uniform(rg_u_y[0], rg_u_y[1])
        pic = gen_rec(dimx,dimy,x0_i,y0_i,uax_i,uay_i,cfu,cbu)
        pic = noize_mixer(pic, noize_level, "gaussian")
        pic = trans_int(pic)
        pic_u.append(pic)
    
    n_col = n_seg
    n_row = 2
    
    #fig=plt.figure(figsize=(n_seg, 2))
    #for i in range(n_seg):
    #    fig.add_subplot(n_row, n_col, i+1)
    #    plt.imshow(pic_01[i])
    #for i in range(n_seg):
    #    fig.add_subplot(n_row, n_col, i+1+n_seg)
    #    plt.imshow(pic_u[i])
    #plt.xlabel('t')
    #plt.show()
    
    d_gap = 20
    lx = d_gap*(n_seg-1)+n_seg*dimx
    ly = d_gap*1 + dimy*2
    a_tot = np.zeros((ly,lx))
    
    #print("lx = ", lx, "ly = ", ly, "a_tot = ", a_tot.shape)
    print("noize level = %.3f"%(noize_level))
    
    for i in range(n_seg):
        x0 = d_gap*(i)+dimx*(i)
        y0 = 0
        a_tot[y0:y0+dimy, x0:x0+dimx] = pic_01[i]
    
    for i in range(n_seg):
        x0 = d_gap*i+dimx*i
        y0 = d_gap + dimy
        a_tot[y0:y0+dimy, x0:x0+dimx] = pic_u[i]
        
    x_tics = []
    x_labels = []
    if(show_t):
        for i in range(n_seg):
            x_tics.append(dimx*0.5+i*(dimx+d_gap))
            x_labels.append("t=%.2f"%(i*1.0/(n_seg-1)))
        
    #y_tics = [0.5*dimy, 0.5*dimy+(dimy+d_gap)]
    #y_labels = ["w. change", "w/o. change"]
    y_tics = []
    
    a_tot = a_tot[0:dimy,:]
    
    fig, ax = plt.subplots()
    #im = ax.imshow(a_tot, cmap='gray')
    ax.axes.set_xticks(x_tics)
    ax.axes.set_yticks(y_tics)
    ax.set_xticklabels(x_labels)
    #ax.set_yticklabels(y_labels)
    im = ax.imshow(a_tot, cmap='gray')
    #plt.xlabel("t")
    #plt.tight_layout()
    plt.savefig(file_name, dpi=400, bbox_inches="tight")
    plt.show()

In [ ]:
## visulaize(demostrate) the data
create_demostration_nosquare(8, 1000, 0.4, 0.10, "./demo.png", rand_pos=True)

In [ ]:
## noize = 0.10, t0=0.4
os.system("rm -r ./image_64_040_2000_noize_010; mkdir ./image_64_040_2000_noize_010/")
os.system("mkdir ./image_64_040_2000_noize_010/dataset_train;")
gen_picture_dataset(64, 64, 1600, 1.0, 1000, 0.4, 0.10, "./image_64_040_2000_noize_010/dataset_train", rand_pos = True)
os.system("mkdir ./image_64_040_2000_noize_010/dataset_test;")
gen_picture_dataset(64, 64, 400, 1.0, 1000, 0.4, 0.10, "./image_64_040_2000_noize_010/dataset_test", rand_pos = True)